In [1]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# Tạo bảng student
cursor.execute("""
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Tạo bảng course
cursor.execute("""
CREATE TABLE IF NOT EXISTS course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Xóa dữ liệu cũ nếu có
cursor.execute("DELETE FROM student;")
cursor.execute("DELETE FROM course;")

# Chèn dữ liệu vào bảng course
cursor.executemany("""
INSERT INTO course (id, course_name) VALUES (?, ?);
""", [(12, 'Giai tich'), (34, 'Thong ke'), (26, 'Tin hoc')])

# Chèn dữ liệu vào bảng student
cursor.executemany("""
INSERT INTO student (student_id, name, class, course_id, score) VALUES (?, ?, ?, ?, ?);
""", [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
])
conn.commit()

# Đóng kết nối
conn.close()


In [14]:

# 🔹 Kết nối đến SQLite
conn = sqlite3.connect("school.db")

# 🔹 Đọc dữ liệu từ SQLite vào Pandas
df_student = pd.read_sql("SELECT * FROM student;", conn)
df_course = pd.read_sql("SELECT * FROM course;", conn)

# 🔹 INNER JOIN (chỉ giữ những dòng có khóa khớp)
df_inner = df_student.merge(df_course, how="inner", left_on="course_id", right_on="id")

# 🔹 LEFT JOIN (giữ tất cả sinh viên, nếu không có khóa khớp thì course_name sẽ là NaN)
df_left = df_student.merge(df_course, how="left", left_on="course_id", right_on="id")

# 🔹 RIGHT JOIN (giữ tất cả khóa học, nếu không có khóa khớp thì student sẽ là NaN)
df_right = df_student.merge(df_course, how="right", left_on="course_id", right_on="id")

# 🔹 FULL OUTER JOIN (kết hợp LEFT và RIGHT JOIN)
df_full = df_student.merge(df_course, how="outer", left_on="course_id", right_on="id")
# 🔹 Đóng kết nối SQLite
conn.close()

# 🔹 In kết quả
print("\n INNER JOIN:")
print(df_inner)

print("\n LEFT JOIN:")
print(df_left)

print("\n RIGHT JOIN:")
print(df_right)

print("\n FULL OUTER JOIN:")
print(df_full)



 INNER JOIN:
   student_id               name     class  course_id  score graduation_date  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7      2025-09-12   
1           2       Tran Thi Lan   Kinh Te         34    9.2      2025-06-12   
2           3       Pham Van Nam  Toan Tin         34    7.9      2025-09-12   
3           7      Bui Tien Dung   Kinh Te         34    9.2      2025-06-12   
4           9     Duong Huu Phuc   Kinh Te         34    7.2      2025-09-12   
5          10       Cao Thi Hanh  May Tinh         34    7.0      2025-09-12   

   id course_name  
0  12   Giai tich  
1  34    Thong ke  
2  34    Thong ke  
3  34    Thong ke  
4  34    Thong ke  
5  34    Thong ke  

 LEFT JOIN:
   student_id               name     class  course_id  score graduation_date  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7      2025-09-12   
1           2       Tran Thi Lan   Kinh Te         34    9.2      2025-06-12   
2           3       Pham Van Nam

In [13]:

print("\n RIGHT JOIN:")
print(df_right)

print("\n FULL OUTER JOIN:")
print(df_full)



 RIGHT JOIN:
   student_id               name     class  course_id  score  id course_name
0         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1         NaN                NaN       NaN        NaN    NaN  26     Tin hoc
2         2.0       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
3         7.0      Bui Tien Dung   Kinh Te       34.0    9.2  34    Thong ke

 FULL OUTER JOIN:
    student_id               name     class  course_id  score    id  \
0          1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7  12.0   
1          2.0       Tran Thi Lan   Kinh Te       34.0    9.2  34.0   
2          7.0      Bui Tien Dung   Kinh Te       34.0    9.2  34.0   
3          3.0       Pham Van Nam  Toan Tin        NaN    7.9   NaN   
4          9.0     Duong Huu Phuc   Kinh Te        NaN    7.2   NaN   
5         10.0       Cao Thi Hanh  May Tinh        NaN    7.0   NaN   
6          4.0     Le Thanh Huyen  Toan Tin       20.0    7.2   NaN   
7          8.0

### 2. Hãy cập nhật những giá trị course_id còn thiếu trong bảng student bằng câu lệnh SQL,
trong đó các giá trị được điền là những giá trị nằm trong bảng course và loại bỏ những
bản ghi tham gia những môn học không tồn tại bảng course. Sau đó hãy cho biết:
- a. Tổng số sinh viên, điểm trung bình của từng lớp.
- b. Tổng số sinh viên, điểm trung bình của từng môn học.
- c. Phân loại thi đua theo số điểm của từng môn học biết:
- i. Điểm TB ≥ 9.0: Xuất sắc.
- ii. 6.0 ≤ Điểm TB ≤ 8.9: Tốt.
- iii. Điểm TB < 6.0: Kém.

In [16]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

#  Cập nhật course_id bị thiếu bằng giá trị hợp lệ từ bảng course
cursor.execute("""
    UPDATE student
    SET course_id = (
        SELECT id FROM course ORDER BY RANDOM() LIMIT 1
    )
    WHERE course_id IS NULL;
""")

#  Loại bỏ sinh viên có course_id không tồn tại trong bảng course
cursor.execute("""
    DELETE FROM student WHERE course_id NOT IN (SELECT id FROM course);
""")

conn.commit()  # Lưu thay đổi vào database

# Hiển thị danh sách tất cả sinh viên kèm thông tin môn học
query_students = """
    SELECT student.student_id, student.name, student.class, 
           student.score, course.course_name
    FROM student
    LEFT JOIN course ON student.course_id = course.id;
"""
df_students = pd.read_sql(query_students, conn)
print("Danh sách tất cả sinh viên:")
print(df_students)

#  Truy vấn tổng số sinh viên và điểm trung bình theo từng lớp
query_class = """
    SELECT class, COUNT(*) AS total_students, ROUND(AVG(score), 2) AS avg_score
    FROM student
    GROUP BY class;
"""
df_class = pd.read_sql(query_class, conn)
print("\n Tổng số sinh viên và điểm trung bình của từng lớp:")
print(df_class)

#  Truy vấn tổng số sinh viên và điểm trung bình theo từng môn học
query_course = """
    SELECT course.course_name, COUNT(student.student_id) AS total_students, 
           ROUND(AVG(student.score), 2) AS avg_score
    FROM student
    JOIN course ON student.course_id = course.id
    GROUP BY course.course_name;
"""
df_course = pd.read_sql(query_course, conn)
print("\n Tổng số sinh viên và điểm trung bình của từng môn học:")
print(df_course)

#  Phân loại thi đua theo điểm trung bình của từng môn học
query_performance = """
    SELECT course.course_name, COUNT(student.student_id) AS total_students, 
        ROUND(AVG(student.score), 2) AS avg_score,
        CASE 
            WHEN AVG(student.score) >= 9.0 THEN 'Xuất sắc'
            WHEN AVG(student.score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
            ELSE 'Kém'
        END AS performance
    FROM student
    JOIN course ON student.course_id = course.id
    GROUP BY course.course_name;
"""
df_performance = pd.read_sql(query_performance, conn)
print("\n Phân loại thi đua theo số điểm của từng môn học:")
print(df_performance)

# Đóng kết nối
conn.close()


Danh sách tất cả sinh viên:
   student_id               name     class  score course_name
0           1  Nguyen Minh Hoang  May Tinh    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te    9.2    Thong ke
2           3       Pham Van Nam  Toan Tin    7.9    Thong ke
3           7      Bui Tien Dung   Kinh Te    9.2    Thong ke
4           9     Duong Huu Phuc   Kinh Te    7.2    Thong ke
5          10       Cao Thi Hanh  May Tinh    7.0    Thong ke

 Tổng số sinh viên và điểm trung bình của từng lớp:
      class  total_students  avg_score
0   Kinh Te               3       8.53
1  May Tinh               2       6.85
2  Toan Tin               1       7.90

 Tổng số sinh viên và điểm trung bình của từng môn học:
  course_name  total_students  avg_score
0   Giai tich               1        6.7
1    Thong ke               5        8.1

 Phân loại thi đua theo số điểm của từng môn học:
  course_name  total_students  avg_score performance
0   Giai tich               1        6.7     

### 3. Hãy xếp hạng sinh viên thông qua:
- a. Điểm số.
- b. Điểm số theo lớp học.
- c. Điểm số theo mã môn học.
- và cho biết top 3 sinh viện đạt thứ hạng cao nhất, top 3 sinh viên đạt thứ hạng thấp nhất theo
từng trường hợp trên.


In [ ]:
import sqlite3
import pandas as pd

# Kết nối SQLite
conn = sqlite3.connect("school.db")

#  Xếp hạng sinh viên theo điểm số toàn bộ
query_rank_overall = """
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (ORDER BY score DESC) AS rank
    FROM student;
"""
df_rank_overall = pd.read_sql(query_rank_overall, conn)

#  Xếp hạng sinh viên theo điểm số trong từng lớp
query_rank_class = """
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank
    FROM student;
"""
df_rank_class = pd.read_sql(query_rank_class, conn)

#  Xếp hạng sinh viên theo điểm số trong từng môn học
query_rank_course = """
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank
    FROM student;
"""
df_rank_course = pd.read_sql(query_rank_course, conn)

#  Lấy top 3 sinh viên cao nhất & thấp nhất theo từng trường hợp
print(" Top 3 sinh viên cao nhất theo toàn bộ điểm số:")
print(df_rank_overall.head(3))  # 3 người có rank cao nhất

print("\n Top 3 sinh viên thấp nhất theo toàn bộ điểm số:")
print(df_rank_overall.tail(3))  # 3 người có rank thấp nhất

print("\n Top 3 sinh viên cao nhất theo từng lớp:")
print(df_rank_class[df_rank_class["rank"] <= 3])

print("\n Top 3 sinh viên thấp nhất theo từng lớp:")
print(df_rank_class.groupby("class").tail(3))  # Lấy cuối 3 người trong mỗi lớp


# Đóng kết nối
conn.close()


🔹 Top 3 sinh viên cao nhất theo toàn bộ điểm số:
   student_id           name     class  course_id  score  rank
0           2   Tran Thi Lan   Kinh Te         34    9.2     1
1           7  Bui Tien Dung   Kinh Te         34    9.2     1
2           3   Pham Van Nam  Toan Tin         34    7.9     3

🔹 Top 3 sinh viên thấp nhất theo toàn bộ điểm số:
   student_id               name     class  course_id  score  rank
3           9     Duong Huu Phuc   Kinh Te         34    7.2     4
4          10       Cao Thi Hanh  May Tinh         34    7.0     5
5           1  Nguyen Minh Hoang  May Tinh         12    6.7     6

🔹 Top 3 sinh viên cao nhất theo từng lớp:
   student_id               name     class  course_id  score  rank
0           2       Tran Thi Lan   Kinh Te         34    9.2     1
1           7      Bui Tien Dung   Kinh Te         34    9.2     1
2           9     Duong Huu Phuc   Kinh Te         34    7.2     3
3          10       Cao Thi Hanh  May Tinh         34    7.0     1
4 

In [10]:
print("\n🔹 Top 3 sinh viên cao nhất theo từng môn học:")
print(df_rank_course[df_rank_course["rank"] <= 3])

print("\n🔹 Top 3 sinh viên thấp nhất theo từng môn học:")
print(df_rank_course.groupby("course_id").tail(3))  # Lấy cuối 3 người trong mỗi môn học

# Đóng kết nối
conn.close()


🔹 Top 3 sinh viên cao nhất theo từng môn học:
   student_id               name     class  course_id  score  rank
0           1  Nguyen Minh Hoang  May Tinh         12    6.7     1
1           2       Tran Thi Lan   Kinh Te         34    9.2     1
2           7      Bui Tien Dung   Kinh Te         34    9.2     1
3           3       Pham Van Nam  Toan Tin         34    7.9     3

🔹 Top 3 sinh viên thấp nhất theo từng môn học:
   student_id               name     class  course_id  score  rank
0           1  Nguyen Minh Hoang  May Tinh         12    6.7     1
3           3       Pham Van Nam  Toan Tin         34    7.9     3
4           9     Duong Huu Phuc   Kinh Te         34    7.2     4
5          10       Cao Thi Hanh  May Tinh         34    7.0     5


### 4

In [12]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Kết nối đến SQLite
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

# 🟢 Thêm cột graduation_date vào bảng student nếu chưa có
cursor.execute("""
    ALTER TABLE student ADD COLUMN graduation_date DATETIME;
""")

# 🟢 Cập nhật giá trị graduation_date dựa trên điểm số
current_date = datetime.now().strftime("%Y-%m-%d")  # Lấy ngày hiện tại

cursor.execute("""
    UPDATE student
    SET graduation_date = 
        CASE 
            WHEN score >= 9.0 THEN DATE('now', '+3 months')
            WHEN score >= 6.0 THEN DATE('now', '+6 months')
            ELSE DATE('now', '+12 months')
        END;
""")

conn.commit()  # Lưu thay đổi

# 🟢 Hiển thị danh sách sinh viên sau khi cập nhật
df_students = pd.read_sql("SELECT * FROM student;", conn)
print("🔹 Danh sách sinh viên sau khi cập nhật thời gian tốt nghiệp:")
print(df_students.to_string())

# Đóng kết nối
conn.close()


🔹 Danh sách sinh viên sau khi cập nhật thời gian tốt nghiệp:
   student_id               name     class  course_id  score graduation_date
0           1  Nguyen Minh Hoang  May Tinh         12    6.7      2025-09-12
1           2       Tran Thi Lan   Kinh Te         34    9.2      2025-06-12
2           3       Pham Van Nam  Toan Tin         34    7.9      2025-09-12
3           7      Bui Tien Dung   Kinh Te         34    9.2      2025-06-12
4           9     Duong Huu Phuc   Kinh Te         34    7.2      2025-09-12
5          10       Cao Thi Hanh  May Tinh         34    7.0      2025-09-12
